In [1]:
import wradlib
import numpy as np
import os
import h5py
import zipfile
import datetime as dt
import pandas as pd

C:\Anaconda3\envs\wradlib1_0\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
for_year = "2016"
rootdir = r"e:\data\radolan\ry"
tmpdir = r"e:\data\radolan\tmp"
h5file = 'ry_%s.hdf5' % for_year
hourlyfile = 'ry_hourly_%s.hdf5' % for_year
tstart = "%s-01-01" % for_year 
tend = "%s-12-31" % for_year
dffile = "exc_%s.csv" % for_year
nx = 900
ny = 900

In [3]:
days = wradlib.util.from_to(dt.datetime.strptime(tstart, "%Y-%m-%d"),
                            dt.datetime.strptime(tend, "%Y-%m-%d"), tdelta=3600*24)
dtimes = wradlib.util.from_to(days[0].strftime("%Y-%m-%d 00:00:00"),
                              days[-1].strftime("%Y-%m-%d 23:55:00"), tdelta=60*5)

hrs = np.array([np.arange(0,24*12, 12), np.arange(12,25*12, 12)]).reshape((-1,2))

exc05 = np.zeros(len(dtimes)) * np.nan
exc1 = np.zeros(len(dtimes)) * np.nan
exc5 = np.zeros(len(dtimes)) * np.nan
exc10 = np.zeros(len(dtimes)) * np.nan

In [4]:
df = pd.DataFrame({"dtime": dtimes, "exc05": exc05, "exc1": exc1, "exc5": exc5, "exc10": exc10})
df = df.set_index('dtime')

In [5]:
with h5py.File(h5file, 'a') as f:
    for day in days:
        h5dir = day.strftime("%Y/%m/%d")
        print(h5dir)
        # Check for zip file and extract all
        zippath = os.path.join(rootdir, day.strftime("%Y/%m/%d/%Y%m%d_RY.zip"))
        if os.path.exists(zippath):
            zf = zipfile.ZipFile(zippath, "r")
            zf.extractall(tmpdir)
            zf.close()
        else:
            print("\tno archive.")
            continue
        # Create dataset if not exists
        data = np.zeros((24*12,900,900)).astype(np.float16) * np.nan
        if not h5dir in f:
            dset = f.create_dataset(h5dir, data.shape, dtype=np.float16, compression="gzip")
        else:
            dset = f[h5dir]
        for i, dtime in enumerate(wradlib.util.from_to(day.strftime("%Y-%m-%d 00:00:00"),
                                                       day.strftime("%Y-%m-%d 23:55:00"), 60*5)):
            ryfile = dtime.strftime("raa01-ry_10000-%y%m%d%H%M-dwd---bin")
            rypath = os.path.join(tmpdir, ryfile)
            try:
                data[i] = wradlib.io.read_radolan_composite(rypath, missing=np.nan)[0]
                os.remove(rypath)
            except FileNotFoundError:
                print("\tnot found: %s" % ryfile)
                continue
            except EOFError:
                print("\tEOFError: %s" % ryfile)
                continue
            except OSError:
                print("\tOSError: %s" % ryfile)
                continue
            df.loc[dtime].exc05 = len(np.where(data[i].ravel() >= 0.5)[0])
            df.loc[dtime].exc1 = len(np.where(data[i].ravel() >= 1.)[0])
            df.loc[dtime].exc5 = len(np.where(data[i].ravel() >= 5.)[0])
            df.loc[dtime].exc10 = len(np.where(data[i].ravel() >= 10.)[0])
        dset[:] = data
            # Hourly RY file
        with h5py.File(hourlyfile, 'a') as hourlyf:
            hdata = np.zeros((24,900,900)).astype(np.float16) * np.nan
            for i, hr in enumerate(hrs):
                hdata[i] = data[hr[0]:hr[1]].sum(axis=0)
            if not h5dir in hourlyf:
                dset = hourlyf.create_dataset(day.strftime("%Y/%m/%d"), data=hdata, compression="gzip")
            else:
                hourlyf[h5dir][:] = hdata

df.to_csv(dffile)

2007/01/01
	no archive.
2007/01/02
	no archive.
2007/01/03
	no archive.
2007/01/04
	no archive.
2007/01/05
	no archive.
2007/01/06
	no archive.
2007/01/07
	no archive.
2007/01/08
	no archive.
2007/01/09
	no archive.
2007/01/10
	no archive.
2007/01/11
	no archive.
2007/01/12
	no archive.
2007/01/13
	no archive.
2007/01/14
	no archive.
2007/01/15
	no archive.
2007/01/16
	no archive.
2007/01/17
	no archive.
2007/01/18
	no archive.
2007/01/19
	no archive.
2007/01/20
	no archive.
2007/01/21
	no archive.
2007/01/22
	no archive.
2007/01/23
	no archive.
2007/01/24
	no archive.
2007/01/25
	no archive.
2007/01/26
	no archive.
2007/01/27
	no archive.
2007/01/28
	no archive.
2007/01/29
	no archive.
2007/01/30
	no archive.
2007/01/31
	no archive.
2007/02/01
	no archive.
2007/02/02
	no archive.
2007/02/03
	no archive.
2007/02/04
	no archive.
2007/02/05
	no archive.
2007/02/06
	no archive.
2007/02/07
	no archive.
2007/02/08
	no archive.
2007/02/09
	no archive.
2007/02/10
	no archive.
2007/02/11
	no a

2007/04/02
	not found: raa01-ry_10000-0704020000-dwd---bin
	not found: raa01-ry_10000-0704020005-dwd---bin
	not found: raa01-ry_10000-0704020010-dwd---bin
	not found: raa01-ry_10000-0704020015-dwd---bin
	not found: raa01-ry_10000-0704020020-dwd---bin
	not found: raa01-ry_10000-0704020025-dwd---bin
	not found: raa01-ry_10000-0704020030-dwd---bin
	not found: raa01-ry_10000-0704020035-dwd---bin
	not found: raa01-ry_10000-0704020040-dwd---bin
	not found: raa01-ry_10000-0704020045-dwd---bin
	not found: raa01-ry_10000-0704020050-dwd---bin
	not found: raa01-ry_10000-0704020055-dwd---bin
	not found: raa01-ry_10000-0704020100-dwd---bin
	not found: raa01-ry_10000-0704020105-dwd---bin
	not found: raa01-ry_10000-0704020110-dwd---bin
	not found: raa01-ry_10000-0704020115-dwd---bin
	not found: raa01-ry_10000-0704020120-dwd---bin
	not found: raa01-ry_10000-0704020125-dwd---bin
	not found: raa01-ry_10000-0704020130-dwd---bin
	not found: raa01-ry_10000-0704020135-dwd---bin
	not found: raa01-ry_10000-07

2007/04/19
	no archive.
2007/04/20
	not found: raa01-ry_10000-0704200000-dwd---bin
	not found: raa01-ry_10000-0704200005-dwd---bin
	not found: raa01-ry_10000-0704200010-dwd---bin
	not found: raa01-ry_10000-0704200015-dwd---bin
	not found: raa01-ry_10000-0704200020-dwd---bin
	not found: raa01-ry_10000-0704200025-dwd---bin
	not found: raa01-ry_10000-0704200030-dwd---bin
	not found: raa01-ry_10000-0704200035-dwd---bin
	not found: raa01-ry_10000-0704200040-dwd---bin
	not found: raa01-ry_10000-0704200045-dwd---bin
	not found: raa01-ry_10000-0704200050-dwd---bin
	not found: raa01-ry_10000-0704200055-dwd---bin
	not found: raa01-ry_10000-0704200100-dwd---bin
	not found: raa01-ry_10000-0704200105-dwd---bin
	not found: raa01-ry_10000-0704200110-dwd---bin
	not found: raa01-ry_10000-0704200115-dwd---bin
	not found: raa01-ry_10000-0704200120-dwd---bin
	not found: raa01-ry_10000-0704200125-dwd---bin
	not found: raa01-ry_10000-0704200130-dwd---bin
	not found: raa01-ry_10000-0704200135-dwd---bin
	not 

2007/04/21
	not found: raa01-ry_10000-0704211800-dwd---bin
	not found: raa01-ry_10000-0704211805-dwd---bin
	not found: raa01-ry_10000-0704211810-dwd---bin
	not found: raa01-ry_10000-0704211815-dwd---bin
	not found: raa01-ry_10000-0704211820-dwd---bin
	not found: raa01-ry_10000-0704211825-dwd---bin
	not found: raa01-ry_10000-0704211830-dwd---bin
	not found: raa01-ry_10000-0704211835-dwd---bin
	not found: raa01-ry_10000-0704211840-dwd---bin
	not found: raa01-ry_10000-0704211845-dwd---bin
	not found: raa01-ry_10000-0704211850-dwd---bin
	not found: raa01-ry_10000-0704211855-dwd---bin
	not found: raa01-ry_10000-0704211900-dwd---bin
	not found: raa01-ry_10000-0704211905-dwd---bin
	not found: raa01-ry_10000-0704211910-dwd---bin
	not found: raa01-ry_10000-0704211915-dwd---bin
	not found: raa01-ry_10000-0704211920-dwd---bin
	not found: raa01-ry_10000-0704211925-dwd---bin
	not found: raa01-ry_10000-0704211930-dwd---bin
	not found: raa01-ry_10000-0704211935-dwd---bin
	not found: raa01-ry_10000-07

2007/04/28
2007/04/29
2007/04/30
2007/05/01
2007/05/02
2007/05/03
2007/05/04
2007/05/05
2007/05/06
2007/05/07
2007/05/08
2007/05/09
2007/05/10
2007/05/11
2007/05/12
2007/05/13
2007/05/14
2007/05/15
2007/05/16
2007/05/17
2007/05/18
2007/05/19
2007/05/20
2007/05/21
2007/05/22
2007/05/23
2007/05/24
	not found: raa01-ry_10000-0705240915-dwd---bin
2007/05/25
2007/05/26
2007/05/27
2007/05/28
2007/05/29
2007/05/30
2007/05/31
2007/06/01
2007/06/02
2007/06/03
2007/06/04
2007/06/05
2007/06/06
2007/06/07
2007/06/08
2007/06/09
2007/06/10
2007/06/11
2007/06/12
2007/06/13
2007/06/14
2007/06/15
2007/06/16
2007/06/17
2007/06/18
2007/06/19
2007/06/20
2007/06/21
2007/06/22
2007/06/23
2007/06/24
2007/06/25
	not found: raa01-ry_10000-0706250615-dwd---bin
	not found: raa01-ry_10000-0706250620-dwd---bin
	not found: raa01-ry_10000-0706250625-dwd---bin
2007/06/26
2007/06/27
	not found: raa01-ry_10000-0706271050-dwd---bin
	not found: raa01-ry_10000-0706271055-dwd---bin
	not found: raa01-ry_10000-0706271200-dwd

	not found: raa01-ry_10000-0707080845-dwd---bin
	not found: raa01-ry_10000-0707080850-dwd---bin
	not found: raa01-ry_10000-0707080855-dwd---bin
	not found: raa01-ry_10000-0707080900-dwd---bin
	not found: raa01-ry_10000-0707080905-dwd---bin
	not found: raa01-ry_10000-0707080910-dwd---bin
	not found: raa01-ry_10000-0707080945-dwd---bin
	not found: raa01-ry_10000-0707080950-dwd---bin
	not found: raa01-ry_10000-0707080955-dwd---bin
	not found: raa01-ry_10000-0707081000-dwd---bin
	not found: raa01-ry_10000-0707081005-dwd---bin
	not found: raa01-ry_10000-0707081010-dwd---bin
	not found: raa01-ry_10000-0707081100-dwd---bin
	not found: raa01-ry_10000-0707081105-dwd---bin
	not found: raa01-ry_10000-0707081110-dwd---bin
	not found: raa01-ry_10000-0707081115-dwd---bin
	not found: raa01-ry_10000-0707081120-dwd---bin
	not found: raa01-ry_10000-0707081125-dwd---bin
	not found: raa01-ry_10000-0707081150-dwd---bin
	not found: raa01-ry_10000-0707081155-dwd---bin
	not found: raa01-ry_10000-0707081200-dw

KeyboardInterrupt: 